In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
features_df = df[['koi_disposition','koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec','koi_period',
                 'koi_slogg','koi_srad']]
features_df

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  \
0          CONFIRMED              0              0              0   
1     FALSE POSITIVE              0              1              0   
2     FALSE POSITIVE              0              1              0   
3          CONFIRMED              0              0              0   
4          CONFIRMED              0              0              0   
...              ...            ...            ...            ...   
6986  FALSE POSITIVE              0              0              0   
6987  FALSE POSITIVE              0              1              1   
6988       CANDIDATE              0              0              0   
6989  FALSE POSITIVE              0              0              1   
6990  FALSE POSITIVE              0              0              1   

      koi_fpflag_ec  koi_period  koi_slogg  koi_srad  
0                 0   54.418383      4.467     0.927  
1                 0   19.899140      4.544     0.868  
2                 0    1.736952      4.564     0.791  
3                 0    2.525592      4.438     1.046  
4                 0    4.134435      4.486     0.972  
...             ...         ...        ...       ...  
6986              1    8.589871      4.296     1.088  
6987              0    0.527699      4.529     0.903  
6988              0    1.739849      4.444     1.031  
6989              0    0.681402      4.447     1.041  
6990              1    4.856035      4.385     1.193  

[6991 rows x 8 columns]

# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = features_df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec','koi_period',
                 'koi_slogg','koi_srad']]
y = features_df['koi_disposition']#.values.reshape(-1,1)
print(X.shape,y.shape)


(6991, 7) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

y_train

6122         CANDIDATE
6370    FALSE POSITIVE
2879    FALSE POSITIVE
107          CONFIRMED
29           CANDIDATE
             ...      
3772    FALSE POSITIVE
5191         CANDIDATE
5226    FALSE POSITIVE
5390    FALSE POSITIVE
860     FALSE POSITIVE
Name: koi_disposition, Length: 5243, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [5]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_scaler

MinMaxScaler()

In [6]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

encoded_y_train

array([0, 2, 2, ..., 2, 2, 2])

In [7]:
from tensorflow.keras.utils import to_categorical

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

# Train the Model



In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=7))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [10]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 0s - loss: 0.3203 - accuracy: 0.7986
Normal Neural Network - Loss: 0.3745361985276438, Accuracy: 0.7986270189285278


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [11]:
# Create the GridSearchCV model
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.793, total=  54.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.3s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.791, total=  47.7s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.780, total=  44.2s
[CV] C=1, gamma=0.0001 ...............................................


In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)